# Agenda

- Assignment feedback
- Handling data
- Generators
- Requests
- Multiprocessing

## Iterators in Python

https://wiki.python.org/moin/Iterator  

### Iterators implement 2 methods:
__iter__() and __next__()  

Iterators minimize memory use sinse each element is lazy loaded.   
Generators are Iterators that can be easily written with a single function (rather than creating a class implementing the Iterator interface)

In [1]:
import random

class RandomIterable:
    """implementation of the iterator protocol with __next__ and __iter__ methods.
    __iter__() returns an iterator (normally the object itself)"""

    def __iter__(self):
        return self
    def __next__(self):
        if random.choice(["go", "go", "go", "go", "stop"]) == "stop":
            raise StopIteration  # signals "the end"
        return 1

In [21]:
[x for x in RandomIterable()]

[1, 1, 1, 1, 1]

In [28]:
iterable = RandomIterable() ## create an instance of iterable
my_iterator = iter(iterable)
try:
    element1 = next(my_iterator)
    element2 = next(my_iterator)
    print(element1,element2) # might throw a StopIterationException before running twice
except StopIteration as e:
    print(type(e))


1 1


### iterators in python

In [4]:
# map
map?
num1_lst = [10,20,30,40]
num2_lst = [4,3,2,1]
add_func = lambda x,y:x+y

result = map(add_func, num1_lst, num2_lst)
print(type(result))
print(result)
print(list(result))

<class 'map'>
[14, 23, 32, 41]


#  An intro to generators

https://wiki.python.org/moin/Generators

In [33]:
def firstn(n):
    """Our first generator that lazy loads each requested element"""
    num = 0
    while num < n:
        yield num
        num += 1

#[x for x in firstn(10)]
fn = firstn(10)
print(next(fn))
print(next(fn))

0
1


In [59]:
lst = list(firstn(10)) # now all elements are loaded in memory, so defies the purpose a bit.
lst

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Generators in Pandas

In [32]:
import pandas as pd
df = pd.read_csv('data/befkbhalderstatkode.csv') 

In [33]:
# show content of the dataframe
df.head()

,AAR,BYDEL,ALDER,STATKODE,PERSONER
0,2015,1,0,5100,614
1,2015,1,0,5104,2
2,2015,1,0,5106,1
3,2015,1,0,5110,1
4,2015,1,0,5120,4


In [34]:
# get a generator from the dataframe to iterate over DataFrame rows as (index, Series) pairs.
g = df.iterrows()
print(type(g))
for idx,row in g:
    if idx < 4:
        print(row,'\n')

<class 'generator'>
AAR         2015
BYDEL          1
ALDER          0
STATKODE    5100
PERSONER     614
Name: 0, dtype: int64 

AAR         2015
BYDEL          1
ALDER          0
STATKODE    5104
PERSONER       2
Name: 1, dtype: int64 

AAR         2015
BYDEL          1
ALDER          0
STATKODE    5106
PERSONER       1
Name: 2, dtype: int64 

AAR         2015
BYDEL          1
ALDER          0
STATKODE    5110
PERSONER       1
Name: 3, dtype: int64 



In [35]:
# using iteritems() instead of iterrows(): Iterates over the DataFrame columns, returning a tuple with the column name and the content as a Series
for x in df.iteritems():
    # print(x)
    pass
print('Printing tuple with 2 values: first value=columnname of first column, second series object with 0-indexed values of the first column')
first_column = next(df.iteritems())
print('column name: {}'.format(first_column[0]),'\n', first_column[1]) 

Printing tuple with 2 values: first value=columnname of first column, second series object with 0-indexed values of the first column
column name: AAR 
 0         2015
1         2015
2         2015
3         2015
4         2015
          ... 
542512    1992
542513    1992
542514    1992
542515    1992
542516    1992
Name: AAR, Length: 542517, dtype: int64


## Using iterators/generators

In [70]:
def check_prime(number):
    for divisor in range(2, int(number ** 0.5) + 1):
        if number % divisor == 0:
            return False
        return True

In [23]:

# import getsizeof from sys module 
from sys import getsizeof 
  
list_comprehension = [i for i in range(100000) if check_prime(i)] 
generator_expression = (i for i in range(100000) if check_prime(i))  
  
#gives size for list comprehension 
x = getsizeof(list_comprehension)  
y = getsizeof(generator_expression)  
print('size of the list in memorey:\t\t{}'.format(x))  
print('size of the generator in memorey:\t{}'.format(y))  


NameError: name 'check_prime' is not defined

#### Put the following into a module: read_print.py and run it

```python 
import os
from memory_profiler import profile

@profile
def read_linewise(path):
    with open(path) as fp:
        for line in fp:
            yield line

@profile
def read_complete(path):
    with open(path) as fp:
        return fp.readlines()

@profile
def print_file_contents_linewise():
    for line in read_linewise('moby_dick.txt'):
        print(line, end='')


if __name__ == '__main__':
    if not os.path.isfile('moby_dick.txt'):
        os.system('wget -O moby_dick.txt http://www.gutenberg.org/files/2701/2701-0.txt')
    print_file_contents_linewise()
    print('\n---------------------')
    read_complete('moby_dick.txt')
```

## profiling the 2 methods
we can see in the third column that line 13 increments the load on memory with 3.1 MB because all data is read at once compared to first profile where the profiled method uses a generator to lazy load each line.

~~~bash
Filename: read_print.py

Line #    Mem usage    Increment   Line Contents
================================================
    15     38.6 MiB     38.6 MiB   @profile
    16                             def print_file_contents():
    17     39.3 MiB      0.3 MiB       for line in read_linewise('moby_dick.txt'):
    18     39.3 MiB      0.0 MiB           print(line, end='')



---------------------
Filename: read_print.py

Line #    Mem usage    Increment   Line Contents
================================================
    10     39.3 MiB     39.3 MiB   @profile
    11                             def read_complete(path):
    12     39.3 MiB      0.0 MiB       with open(path) as fp:
    13     42.4 MiB      3.1 MiB           return fp.readlines()
    

#### The Increment column is where we can see the added load on memory from particular code line



## Exercise create generator
Create a generator function that can take a list of names as parameter and return each name. 
Get approved unisex names here: 

`wget -O unisex_navne.xls https://ast.dk/_namesdb/export/names?format=xls&gendermask=4`

In [10]:
try:
    !wget -O unisex_navne.xls "https://ast.dk/_namesdb/export/names?format=xls&gendermask=4"
except e:
    print(e)
    


--2022-03-08 14:07:32--  https://ast.dk/_namesdb/export/names?format=xls&gendermask=4
Resolving ast.dk (ast.dk)... 77.221.238.185
Connecting to ast.dk (ast.dk)|77.221.238.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50688 (50K) [application/vnd.ms-excel]
Saving to: ‘unisex_navne.xls’

unisex_navne.xls    100%[===================>]  49.50K  --.-KB/s    in 0.02s   

2022-03-08 14:07:32 (2.48 MB/s) - ‘unisex_navne.xls’ saved [50688/50688]



In [58]:
import xlrd


file = xlrd.open_workbook("unisex_navne.xls")

sheet1 = file.sheet_by_index(0)
for row in sheet1.get_rows():
    print(row[0].value)
    print(getsizeof(row))

x = getsizeof(b)


print(x)




Abel
104
Abiola
104
Acelya
104
Ada
104
Adama
104
Adan
104
Adar
104
Addis
104
Addison
104
Aden
104
Adi
104
Adian
104
Adita
104
Aezha
104
Afnan
104
Agam
104
Agne
104
Ahsen
104
Ahura
104
Aidan
104
Aiden
104
Aiko
104
Aimal
104
Aiman
104
Aisosa
104
Ajnur
104
Ajo
104
Aki
104
Akila
104
Akira
104
Akisooq
104
Akram
104
Akshini
104
Al
104
Ala
104
Alae
104
Alaz
104
Albi
104
Alem
104
Alex
104
Alexis
104
Alfie
104
Alham
104
Alija
104
Alin
104
Alix
104
Alla
104
Alli
104
Almas
104
Alpha
104
Altaf
104
Alvild
104
Alaa
104
Amal
104
Aman
104
Amani
104
Amar
104
Amaru
104
Amel
104
Amen
104
Ami
104
Amine
104
Amran
104
An
104
Anan
104
Ananda
104
Anava
104
Anda
104
Andrea
104
Andy
104
Anele
104
Ange
104
Angel
104
Anh
104
Ani
104
Anis
104
Anmol
104
Anne
104
Antonie
104
Anuk
104
Anur
104
Anwar
104
Aputsiaq
104
Ara
104
Arda
104
Arden
104
Areg
104
Ari
104
Aria
104
Arian
104
Ariel
104
Arin
104
Ariya
104
Arjin
104
Armani
104
Arshia
104
Arta
104
Arya
104
Asa
104
Ashley
104
Asil
104
Aska
104
Aslaug
104
Assana
104
Ast

104
Wan
104
Warin
104
Wei
104
Wen
104
Wilde
104
Willi
104
Willow
104
Win
104
Wing
104
Xi
104
Xian
104
Xiang
104
Xiao
104
Xiaoye
104
Xin
104
Xing
104
Xuan
104
Yahya
104
Yali
104
Yan
104
Yang
104
Yao
104
Yarin
104
Yee
104
Yi
104
Yichen
104
Yin
104
Yiran
104
Yixuan
104
Yo
104
Yoel
104
Yon
104
Yona
104
Yong
104
Yoon
104
Young
104
Yu
104
Yuan
104
Yue
104
Yuki
104
Yumna
104
Yuna
104
Yuri
104
Yüksel
104
Zacharia
104
Zana
104
Zappa
104
Zayed
104
Zaza
104
Ze
104
Zein
104
Zephyr
104
Zhen
104
Zi
104
Zia
104
Zichen
104
Zidan
104
Ziggi
104
Ziggy
104
Zihan
104
Zilin
104
Zin
104
Ziyuan
104
Zulal
104
Aaria
104
Aarya
104
64


## The `%timeit` magic
Calculate execution time of a Python statement or expression
Runs the code 10.000 times and collect statistics (mean + standard deviation)

In [5]:
import time
def waiting():
    time.sleep(1)
    
%timeit waiting()

1 s ± 353 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
# line magic function
%timeit sum(range(0, 1000)) 

10.9 µs ± 691 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [40]:
%timeit sum(list(range(0,1000)))

18.4 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [41]:
%%timeit #must be first line in cell
#cell magic function

sum(range(0, 100)) 
sum(range(0, 100)) 

1.88 µs ± 60.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Exercise python modules:

1. make 2 files, a main file and a module file, 'called test_my_module.py' and 'get_names.py' respectively.
2. in the module file write a function with a generator, that can serve one name at a time (like you created in the last lesson)
3. execute the function in the module file and test run it from cli with: `python get_names.py`
4. in the main file implement a function that can take a number and return that many names (using the module you made).
5. make sure that test_my_module.py can be run directly and that when running test_my_module, no top level code from get_names will run.